# Create an AI pipeline for quiz generation, using langgraph

## Overview

Create a quiz about a news article, using GenAI, RAG, and LangGraph. 

## AI pipeline

The pipeline includes:
- The user specifies a **topic** of interest
- Search for corresponding news articles in the **Reuters dataset**
- The user selects 1 from the top-3 new articles
- Generate quiz with a **prompt template**
- The generated quiz will have an output format according to our specifications

![AI pipeline](ai-pipeline.png "AI pipeline for quiz generation")

In [ ]:
# Install necessary dependencies
# - this takes ~3 minutes, give it some patience
# - the imports can show error messages, you can ignore them

!pip install unsloth
!pip install langchain
!pip install langgraph
!pip install langchain_huggingface

In [ ]:
# Do all necessary imports

from typing import List, Dict
from typing_extensions import TypedDict
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langgraph.types import Command, interrupt
from langgraph.graph import START, StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_huggingface import HuggingFaceEmbeddings
from IPython.display import Image, display
from datasets import load_dataset
import pandas as pd
from unsloth import FastLanguageModel
from transformers import PreTrainedTokenizer, PreTrainedModel

# Prepare LLM for quiz generation

In [ ]:
# Instantiate unsloth model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
)
FastLanguageModel.for_inference(model)

In [ ]:
# Define function for LLM inference

def llm_inference(
        messages: List[Dict],
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer
) -> str:
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    input_tokens = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")
    input_len = len(input_tokens.tokens())
    output_tokens = model.generate(**input_tokens)
    output_clipped = output_tokens[:, input_len:-1]
    result = tokenizer.batch_decode(output_clipped)
    return result[0]

# Define prompt template for quiz generation

In [ ]:
def create_messages(news_article: str) -> List[Dict]:
    messages = [
        {
            "role": "system",
            "content": """Please generate one multiple choice quiz for one provided news article.
            
            The quiz should have the following format:
            
            [Question]
            
            [Choice 1]
            [Choice 2]
            [Choice 3]
            
            [Solution]
            """,
        },
        {
            "role": "user",
            "content": f"Here is the news article: {news_article}"
        }
    ]
    return messages

In [ ]:
news_article = "U.S. Agriculture Secretary Richard Lyng said he would not agree to an extension of the 18-month whole dairy herd buyout program set to expire later this year. Speaking at the Agriculture Department to representatives of the U.S. National Cattlemen\'s Association, Lyng said some dairymen asked the program be extended. But he said the Reagan administration, which opposed the whole herd buyout program in the 1985 farm bill, would not agree to an extension. The program begun in early 1986, is to be completed this summer. U.S. cattlemen bitterly opposed the scheme, complaining that increased dairy cow slaughter drove cattle prices down last year. Reuter"
messages = create_messages(news_article)
llm_response = llm_inference(messages, model, tokenizer)
print(f"LLM response:\n{llm_response}")

## TODO: Adjust prompt

Try to adjust the prompt template, in order to get the following results:
- A quiz with more multiple choice options
- A more tricky quiz
- ... think of another adjustment ...

In [ ]:
# TODO: adjust prompt template
# <your code goes here>

## Load Reuters dataset

Huggingface dataset: https://huggingface.co/datasets/ucirvine/reuters21578

In [ ]:
# Load dataset from Huggingface - if asked to run custom code, type "y" for YES.
reuters_ds = load_dataset('ucirvine/reuters21578','ModHayes')
news_raw = reuters_ds["train"].to_pandas()
print(f"Loaded {len(news_raw)} news articles.")

## Preprocess articles

In [ ]:
# Merge title and text, drop unnecessary columns
news_raw["title_and_text"] = news_raw['title'] + ' | ' + news_raw['text']
news = news_raw[["title_and_text", "date", "places"]]

In [ ]:
# Clean up text, remove unnecessary characters
pd.options.mode.chained_assignment = None
news["title_and_text"] = news.apply(lambda x: x["title_and_text"].replace("\\n", " "), axis=1)
news["title_and_text"] = news.apply(lambda x: x["title_and_text"].replace("\\\"", "\""), axis=1)
news["title_and_text"] = news.apply(lambda x: " ".join(x["title_and_text"].split()), axis=1)
news.head()


In [ ]:
# Setup RAG vector store
# - This can take ~ 1 minute, give it some patience
texts_to_encode = news['title_and_text'].to_list()
embedder = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)
vectorstore = InMemoryVectorStore.from_texts(texts=texts_to_encode, embedding=embedder)

In [ ]:
# Search articles about a given topic
result = vectorstore.similarity_search("An article on agriculture", k=3)
print(result)

# TODO: Search for articles on another topic

In [ ]:
# TODO: Search for articles on another subject
# Potential subjects include: "Japan", "Taxes", etc.
# < your code goes here >

# Create graph with LangGraph

In [ ]:
class State(TypedDict):
    topic: str
    ragged_documents: List[Document]
    selected_document: Document
    quiz: str

def retrieve(state: State):
    retrieved_docs = vectorstore.similarity_search(state["topic"], k=3)
    return {"ragged_documents": retrieved_docs}

def human_feedback(state: State):
    article_selection = interrupt("Let user choose article")
    selected_document=state["ragged_documents"][int(article_selection)]
    return {"selected_document": selected_document}

def generate(state: State):
    news_article = state["selected_document"].page_content
    messages = create_messages(news_article)
    llm_response = llm_inference(messages, model, tokenizer)
    return {"quiz": llm_response}

builder = StateGraph(State)
builder.add_node("retrieve", retrieve)
builder.add_node("human_feedback", human_feedback)
builder.add_node("generate", generate)
builder.add_edge(START, "retrieve")
builder.add_edge("retrieve", "human_feedback")
builder.add_edge("human_feedback", "generate")
builder.add_edge("generate", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
thread = {"configurable": {"thread_id": "1"}}

display(Image(graph.get_graph().draw_mermaid_png()))

# Run the graph

In [ ]:
# Run the graph from the start, until user selection step
topic = input("Please select your topic: ")
initial_input = {"topic": topic}
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    pass
# Display article options
article_options=graph.get_state(config=thread).values['ragged_documents']
print("Article candidates:")
for id, doc in enumerate(article_options):
    content = doc.page_content
    print(f"[{id}] {content}")

In [ ]:
# Get human feedback
human_feedback = input("Please select which article you'd like to use [0,1,2]: ")

# Continue the graph execution
for event in graph.stream(
        Command(resume=human_feedback), thread, stream_mode="updates"
):
    pass

In [ ]:
# Show final quiz
quiz=graph.get_state(config=thread).values['quiz']
print(quiz)

# Adjust quiz

In [ ]:
# TODO: Make the quiz more entertaining. Add the following:
# The final output should include the year of the news article.
# You'll need to go back to the data preparation step, and make sure the year is included in the metadata of the LangChain documents.
# Additionally, show the original news article above the quiz, when displaying the quiz to the user.
# < your code goes here >